In [1]:
import pandas as pd 

import pytrec_eval
import pyterrier as pt 
pt.init()

PyTerrier 0.9.1 has loaded Terrier 5.7 (built by craigm on 2022-11-10 18:30) and terrier-helper 0.0.7

No etc/terrier.properties, using terrier.default.properties for bootstrap configuration.


In [2]:
# df_dense = pd.read_csv("dense_retrieval_result.csv", names = ["qid","docno","score"], sep = "\t")

# def sort_group(group):
#     new = group[["docno","score"]].sort_values(by = "score", ascending = False)
#     new["q0"] = ["q0"]*len(group)
#     new["run_name"] = ["dense"]*len(group)
#     new["rank"] = range(1,len(new) + 1)
#     return new

# df_new = df_dense.groupby("qid").apply(sort_group).reset_index()
# df_new.head()

,qid,level_1,docno,score,q0,run_name,rank
0,1,922,31715818,102.647125,q0,dense,1
1,1,801,4346436,75.595703,q0,dense,2
2,1,713,20101846,73.498718,q0,dense,3
3,1,326,4423327,73.491776,q0,dense,4
4,1,900,502797,72.541901,q0,dense,5


In [4]:
# df_new[["qid","q0","docno","rank","score","run_name"]].to_csv("dense_run_scifact.txt",index = None,header=None, sep = "\t")
df_new = pd.read_csv("dense_run_scifact.txt",names=["qid","q0","docno","rank","score","run_name"], sep = "\t")
df_new.head()

,qid,q0,docno,rank,score,run_name
0,1,q0,31715818,1,102.647125,dense
1,1,q0,4346436,2,75.595703,dense
2,1,q0,20101846,3,73.498718,dense
3,1,q0,4423327,4,73.491776,dense
4,1,q0,502797,5,72.541901,dense


In [5]:
# 检索
index = pt.IndexFactory.of(
    "/projects/futhark1/data/wzm289/code/GAR/gar/indices/beir_scifact"
)
print(index.getCollectionStatistics().toString())
bm25 = pt.BatchRetrieve(index, wmodel="BM25")
train_dataset = pt.get_dataset("irds:beir/scifact/train")
train_qrel = train_dataset.get_qrels()
# querys = train_dataset.get_topics("text")
test_dataset = pt.get_dataset("irds:beir/scifact/test")
test_qrel = test_dataset.get_qrels()
test_querys = test_dataset.get_topics("text")
print(test_qrel)

Number of documents: 5183
Number of terms: 25508
Number of postings: 442057
Number of fields: 2
Number of tokens: 745811
Field names: [text, title]
Positions:   false

      qid     docno  label iteration
0       1  31715818      1         0
1       3  14717500      1         0
2       5  13734012      1         0
3      13   1606628      1         0
4      36   5152028      1         0
..    ...       ...    ...       ...
334  1379  17450673      1         0
335  1382  17755060      1         0
336  1385    306006      1         0
337  1389  23895668      1         0
338  1395  17717391      1         0

[339 rows x 4 columns]


In [6]:
test_qrel = pd.read_csv("../scifact/qrels/test.tsv",sep = "\t")
test_qrel = test_qrel.astype({"score": "int32","query-id":str,"corpus-id":str})
test_qrel

,query-id,corpus-id,score
0,1,31715818,1
1,3,14717500,1
2,5,13734012,1
3,13,1606628,1
4,36,5152028,1
...,...,...,...
334,1379,17450673,1
335,1382,17755060,1
336,1385,306006,1
337,1389,23895668,1


In [20]:


def evaluation(path):
    # if stu_id == 'mlg109':
    #     return
    # Load run
    try:

        with open(path, "r") as f_run:
            tf_run = pytrec_eval.parse_run(f_run)
        qrels_dict = dict()
        for _, r in test_qrel.iterrows():
            qid, docno, label = r
            if qid not in qrels_dict:
                qrels_dict[qid] = dict()
            qrels_dict[qid][docno] = int(label)

        metrics = {"map", "ndcg_cut_10"}
        evaluator = pytrec_eval.RelevanceEvaluator(qrels_dict, metrics)

        # test
        tf_evals = evaluator.evaluate(tf_run)
        tf_metric2vals = {m: [] for m in metrics}
        for q, d in tf_evals.items():
            for m, val in d.items():
                tf_metric2vals[m].append(val)
        
        for m in metrics:
            score = pytrec_eval.compute_aggregated_measure(m, tf_metric2vals[m])
            print(m, "\t", score)

        # fout.write(",".join(write_list) + "\n")
        # fout.flush()
        return tf_evals.keys(), tf_metric2vals['map'],tf_metric2vals['ndcg_cut_10']
    except Exception as e:
        print(e)
qid_list,map_list_dense, ndcg_list_dense = evaluation("dense_run_scifact.txt")

ndcg_cut_10 	 0.7278507119908757
map 	 0.7057529284388354


# Fusion



In [9]:
top = 10
dense_retrieval = df_new[["qid","q0","docno","rank","score","run_name"]]
dense_retrieval = dense_retrieval.groupby("qid").head(top).reset_index()

ndcg_reward = pd.read_csv("/projects/futhark1/data/wzm289/code/trl/result_run/ndcg_cut_10_run.txt",names = ["qid","q0","docno","rank","score","run_name"],sep = "\t")
ndcg_reward = ndcg_reward.groupby("qid").head(top).reset_index()

In [10]:
concat_df = pd.concat([dense_retrieval,ndcg_reward]).sort_index(kind="mergesort").reset_index(drop=True)

In [11]:
def unique_passage(group):
    docno = group["docno"].unique()

    return pd.DataFrame(
        {
            "docno": docno,
            "Q0": ["Q0"] * len(docno),
            "score": list(range(len(docno)))[::-1],
            "tag": ["Anserini"] * len(docno),
        }
    )

concat_df = concat_df.groupby(["qid"]).apply(unique_passage).reset_index()

concat_df

,qid,level_1,docno,Q0,score,tag
0,1,0,31715818,Q0,17,Anserini
1,1,1,4346436,Q0,16,Anserini
2,1,2,21456232,Q0,15,Anserini
3,1,3,20101846,Q0,14,Anserini
4,1,4,3770726,Q0,13,Anserini
...,...,...,...,...,...,...
5327,1395,12,40349336,Q0,4,Anserini
5328,1395,13,46617075,Q0,3,Anserini
5329,1395,14,24347647,Q0,2,Anserini
5330,1395,15,31293581,Q0,1,Anserini


In [53]:
concat_df[["qid", "Q0", "docno", "level_1", "score", "tag"]].to_csv("fusion_ndcg.txt",sep = "\t",index = None, header = None)

In [22]:
qids, map_list_fusion, ndcg_list_fusion = evaluation("fusion_ndcg.txt")

ndcg_cut_10 	 0.7858617247063453
map 	 0.744321164946165


# TT test

In [16]:
import scipy.stats
print(scipy.stats.ttest_rel(map_list_dense, map_list_fusion))

Ttest_relResult(statistic=-4.476532971994697, pvalue=1.0801550824887567e-05)


In [17]:
print(scipy.stats.ttest_rel(ndcg_list_dense, ndcg_list_fusion))

Ttest_relResult(statistic=-5.271280406017071, pvalue=2.6001216003136786e-07)


In [24]:
print(scipy.stats.wilcoxon(map_list_dense, map_list_fusion))

WilcoxonResult(statistic=1656.5, pvalue=0.002820618820028983)
